# Test feature_extraction131(clip_data)

In [4]:
# Importing the Libraries
import os
import platform
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import pathlib
import pickle
from itertools import product
from scipy.stats import skew, kurtosis, pearsonr, iqr, zscore
from scipy.signal import butter, welch, filtfilt, resample
import time
import re
import copy
# import nolds
from sklearn import preprocessing

# from PreprocessFcns import *

%matplotlib inline

In [5]:
if platform.system() == 'Windows':
    if platform.release() == '10':
        path = r'//FS2.smpp.local\RTO\Inpatient Sensors -Stroke\Data\biostamp_data\controls'
        folder_path = r'//FS2.smpp.local\RTO\Inpatient Sensors -Stroke\Data\biostamp_data'
        dict_path = r'//FS2.smpp.local\RTO\Inpatient Sensors -Stroke\Data\biostamp_data\Data_dict'
        features_path = r'//FS2.smpp.local\RTO\Inpatient Sensors -Stroke\Data\biostamp_data\FeatureMatrix'

# Play with HC02 dataset

In [6]:
# use context manager to load pickle file
dict_path2 = r'//FS2.smpp.local\RTO\Inpatient Sensors -Stroke\Data\biostamp_data\Data_dict'
subj = 'HC02'
filename = os.path.join(dict_path2, subj + 'dict.pkl')
with open(filename,'rb') as filename:
    act_dict = pickle.load(filename)

# Extract full recording
DataPreprocessor2_wTime.ipynb

In [7]:
# choose task, location
task = 'WALKING'
loc = 'sacrum'
# sensor = 'accel'
# trial = 1

In [8]:
#clipsize=0 extracts full recordings
clip_data = gen_clips_merged(act_dict,task,loc,clipsize=0,verbose=True)

WALKING  sensortype = accel - trial 0
WALKING  sensortype = gyro - trial 0
WALKING  sensortype = accel - trial 1
WALKING  sensortype = gyro - trial 1
WALKING  sensortype = accel - trial 2
WALKING  sensortype = gyro - trial 2
WALKING  sensortype = accel - trial 3
WALKING  sensortype = gyro - trial 3


# Feature extraction using feature_extraction131()

In [9]:
copy_clip = clip_data.copy()

In [12]:
feature_extraction131(copy_clip)

In [19]:
copy_clip[0]['accel']['features']

,meanX_acc,meanY_acc,meanZ_acc,rangeX_acc,rangeY_acc,rangeZ_acc,iqrX_acc,iqrY_acc,iqrZ_acc,stddev_X_acc,...,meanpower_bin11_z_acc,meanpower_bin12_z_acc,meanpower_bin13_z_acc,meanpower_bin14_z_acc,meanpower_bin15_z_acc,meanpower_bin16_z_acc,meanpower_bin17_z_acc,meanpower_bin18_z_acc,meanpower_bin19_z_acc,meanpower_bin20_z_acc
0,-0.028107,0.981632,0.282775,0.717174,0.941787,0.74696,0.155093,0.272526,0.214237,0.117967,...,0.00056,0.028708,0.003797,0.000316,0.000647,0.006482,0.002114,0.000556,0.000478,0.000332


In [20]:
copy_clip[0]['gyro']['features']

,meanX_gyr,meanY_gyr,meanZ_gyr,rangeX_gyr,rangeY_gyr,rangeZ_gyr,iqrX_gyr,iqrY_gyr,iqrZ_gyr,stddev_X_gyr,...,meanpower_bin11_z_gyr,meanpower_bin12_z_gyr,meanpower_bin13_z_gyr,meanpower_bin14_z_gyr,meanpower_bin15_z_gyr,meanpower_bin16_z_gyr,meanpower_bin17_z_gyr,meanpower_bin18_z_gyr,meanpower_bin19_z_gyr,meanpower_bin20_z_gyr
0,1.951188,-10.61773,-1.893648,57.007706,189.88327,180.422668,16.876478,36.652172,32.227055,11.32982,...,7.854145,4.565918,1.258525,4.35949,1.32924,0.692294,1.598647,24.72431,9.249282,1.61767


# Functions

In [1]:
# changed len tolerance to 0.95 since some clips are < 10 sec
def gen_clips_merged(act_dict,task,location,clipsize=10000,overlap=0.9,verbose=False,startTS=0,endTS=1,
              len_tol=0.95,resample=False):
    """
    Extract clips and merge into 1 clip for accelerometer and gyro data (allows selecting start and end fraction)
    len_tol is the % of the intended clipsize below which clip is not used
    
    :param clipsize 10000 = 10 sec
    :param overlap=0.9 for 90% overlap b/n clips
    :param len_tol=1.0, want complete 10 sec clips
    
    """
    
    clip_data = {} #the dictionary with clips

    for trial in act_dict[task].keys():
        clip_data[trial] = {}

        for s in ['accel','gyro']:

            if verbose:
                print(task,' sensortype = %s - trial %d'%(s,trial))
            #create clips and store in a list
            rawdata = act_dict[task][trial][location][s]
            if rawdata.empty is True: #skip if no data for current sensor
                continue
            #reindex time (relative to start)
            idx = rawdata.index
            idx = idx-idx[0]
            rawdata.index = idx
            #choose to create clips only on a fraction of the data (0<[startTS,endTS]<1)
            if (startTS > 0) | (endTS < 1):
                rawdata = rawdata.iloc[round(startTS*len(rawdata)):round(endTS*len(rawdata)),:]
                #reindex time (relative to start)
                idx = rawdata.index
                idx = idx-idx[0]
                rawdata.index = idx
            #create clips data
            deltat = np.median(np.diff(rawdata.index))
            clips = []
            #use entire recording
            if clipsize == 0:
                clips.append(rawdata)
            #take clips
            else:
                idx = np.arange(0,rawdata.index[-1],clipsize*(1-overlap))
                for i in idx:
                    c = rawdata[(rawdata.index>=i) & (rawdata.index<i+clipsize)]
                    #keep/append clips that are 10 sec, else discard those that don't meet length
                    #tolerance
                    ## clip len tolerance > 9.5 sec
                    if len(c) > len_tol*int(clipsize/deltat):
                        # try concat instead of append to make one list
                        # check index, if increases like 0, 32, 64, etc then great, otherwise
                        # reindex c before extending?
                        clips.append(c)

            # merge all clips into one
            # cycle through each list element, reindex
            if len(clips)>1:
#                 finalclip = []
                for x in range(len(clips)):
                    if x==0:
                        clips.append(clips[0])
                    else:
                        # reindex
                        idx2 = clips[x].index
                        idx2 = idx2-idx2[0]+32+clips[x-1].index[-1]
                        clips[x].index = idx2
                        # merge into list of dataframes
                        clips.append(clips[x])
    
            # merge into one dataframe
            try:
                oneclip = pd.concat(clips)
            except:
                print('Check len_tol in gen_clips_merged function.')
            # reset clips
            clips = []
            clips.append(oneclip)
    
            #store clip length
            #store the length of each clip
            clip_len = [clips[c].index[-1]-clips[c].index[0] for c in range(len(clips))] 
            #assemble in dict
            clip_data[trial][s] = {'data':clips, 'clip_len':clip_len}

    return clip_data

In [2]:
def power_spectra_welch_axis(rawdata,fm,fM):
    """Compute PSD on each axis then combine into a dataframe"""
    x = rawdata.iloc[:,0]
    y = rawdata.iloc[:,1]
    z = rawdata.iloc[:,2]
    n = len(x) #number of samples in clip
    Fs = np.mean(1/(np.diff(x.index)/1000)) #sampling rate in clip
    fx,Pxx_denX = welch(x,Fs,nperseg=min(256,n))
    fy,Pxx_denY = welch(y,Fs,nperseg=min(256,n))
    fz,Pxx_denZ = welch(z,Fs,nperseg=min(256,n))
    #return PSD in desired interval of freq
    inds = (fx<=fM)&(fx>=fm)
    f=fx[inds]
    Pxx_denX=Pxx_denX[inds]
    Pxx_denY=Pxx_denY[inds]
    Pxx_denZ=Pxx_denZ[inds]
    Pxx_den = {'PSD_X':Pxx_denX,'PSD_Y':Pxx_denY,'PSD_Z':Pxx_denY}
    Pxxdf = pd.DataFrame(data=Pxx_den,index=f)

    return Pxxdf

In [11]:
def feature_extraction131(clip_data):
    """
    Extract features from both sensors (accel and gyro) for current clips and trials
    Input: dictionary of clips from each subject
    Output: feature matrix from all clips from given subject and scores for each clip
    Column names separate for acc and gyro data.
    """
    
    features_list = ['meanX','meanY','meanZ','rangeX','rangeY','rangeZ','iqrX','iqrY','iqrZ',
                     'stddev_X','stddev_Y','stddev_Z','skewX','skewY','skewZ','kurtX','kurtY','kurtZ',
                     'hist1_X','hist2_X','hist3_X','hist4_X',
                     'hist1_Y','hist2_Y','hist3_Y','hist4_Y',
                     'hist1_Z','hist2_Z','hist3_Z','hist4_Z',
                     #Moments of derivative: mean, SD, skew, kurtosis
                     'mean_derivative_x','mean_derivative_y','mean_derivative_z',
                     'std_derivative_x','std_derivative_y','std_derivative_z',
                     'skew_derivative_x','skew_derivative_y','skew_derivative_z',
                     'kurt_derivative_x','kurt_derivative_y','kurt_derivative_z',
                     'mean_squared_norm','sum_stddev',
                     'xcorr_XY','xcorr_XZ','xcorr_YZ',
                     'crossprod_raw_xy','crossprod_raw_xz','crossprod_raw_yz',
                     'crossprod_norm_xy','crossprod_norm_xz','crossprod_norm_yz',
                     'abs_crossprod_raw_xy','abs_crossprod_raw_xz','abs_crossprod_raw_yz',
                     'abs_crossprod_norm_xy','abs_crossprod_norm_xz','abs_crossprod_norm_yz',
                     'PSD_mean_X','PSD_mean_Y','PSD_mean_Z',
                     'PSD_std_X','PSD_std_Y','PSD_std_Z',
                     'PSD_skew_X','PSD_skew_Y','PSD_skew_Z',
                     'PSD_kur_X','PSD_kur_Y','PSD_kur_Z',
                     # mean power 20 bins
                     # x axis
                     'meanpower_bin1_x','meanpower_bin2_x','meanpower_bin3_x','meanpower_bin4_x',
                     'meanpower_bin5_x','meanpower_bin6_x','meanpower_bin7_x','meanpower_bin8_x',
                     'meanpower_bin9_x','meanpower_bin10_x','meanpower_bin11_x','meanpower_bin12_x',
                     'meanpower_bin13_x','meanpower_bin14_x','meanpower_bin15_x','meanpower_bin16_x',
                     'meanpower_bin17_x','meanpower_bin18_x','meanpower_bin19_x','meanpower_bin20_x',
                     # y axis
                     'meanpower_bin1_y','meanpower_bin2_y','meanpower_bin3_y','meanpower_bin4_y',
                     'meanpower_bin5_y','meanpower_bin6_y','meanpower_bin7_y','meanpower_bin8_y',
                     'meanpower_bin9_y','meanpower_bin10_y','meanpower_bin11_y','meanpower_bin12_y',
                     'meanpower_bin13_y','meanpower_bin14_y','meanpower_bin15_y','meanpower_bin16_y',
                     'meanpower_bin17_y','meanpower_bin18_y','meanpower_bin19_y','meanpower_bin20_y',
                     # z axis
                     'meanpower_bin1_z','meanpower_bin2_z','meanpower_bin3_z','meanpower_bin4_z',
                     'meanpower_bin5_z','meanpower_bin6_z','meanpower_bin7_z','meanpower_bin8_z',
                     'meanpower_bin9_z','meanpower_bin10_z','meanpower_bin11_z','meanpower_bin12_z',
                     'meanpower_bin13_z','meanpower_bin14_z','meanpower_bin15_z','meanpower_bin16_z',
                     'meanpower_bin17_z','meanpower_bin18_z','meanpower_bin19_z','meanpower_bin20_z',]
    acclist = [s + '_acc' for s in features_list]
    gyrlist = [s + '_gyr' for s in features_list]

    for trial in clip_data.keys():

        for sensor in clip_data[trial].keys():

            #cycle through all clips for current trial and save dataframe of features for current trial and sensor
            features = []
            for c in range(len(clip_data[trial][sensor]['data'])):
                rawdata = clip_data[trial][sensor]['data'][c]

                ######################
                # Time domain features
                ######################
                #range on each axis
                min_xyz = np.min(rawdata,axis=0)
                max_xyz = np.max(rawdata,axis=0)
                r = np.asarray(max_xyz-min_xyz)

                #Moments on each axis - mean, std dev, skew, kurtosis
                mean = np.asarray(np.mean(rawdata,axis=0))
                std = np.asarray(np.std(rawdata,axis=0))
                sk = skew(rawdata)
                kurt = kurtosis(rawdata)

                #Cross-correlation between axes pairs
# Which mode?
#                 xcorr_xy = np.correlate(rawdata.iloc[:,0],rawdata.iloc[:,1],mode='same')
#                 xcorr_xz = np.correlate(rawdata.iloc[:,0],rawdata.iloc[:,2],mode='same')
#                 xcorr_yz = np.correlate(rawdata.iloc[:,1],rawdata.iloc[:,2],mode='same')
                xcorr_xy = np.correlate(rawdata.iloc[:,0],rawdata.iloc[:,1])
                xcorr_xz = np.correlate(rawdata.iloc[:,0],rawdata.iloc[:,2])
                xcorr_yz = np.correlate(rawdata.iloc[:,1],rawdata.iloc[:,2])
#                 xcorr = np.array([xcorr_xy, xcorr_xz, xcorr_yz])
                xcorr = np.array([xcorr_xy[0], xcorr_xz[0], xcorr_yz[0]])

################
# Added features
################
# after adding new features...
# check names in features_list at top
# and concatenate at end

                # interquartile range
                iqrange = iqr(rawdata,axis=0)
                # histogram of z-score values
                hist_z_scores_x = np.histogram(zscore(rawdata.iloc[:,0],axis=0), bins=4, range=(-2,2))
                hist_z_scores_y = np.histogram(zscore(rawdata.iloc[:,1],axis=0), bins=4, range=(-2,2))
                hist_z_scores_z = np.histogram(zscore(rawdata.iloc[:,2],axis=0), bins=4, range=(-2,2))
#                 hist_z_scores = np.array([hist_z_scores_x, hist_z_scores_y, hist_z_scores_z])
                hist_z_scores = np.concatenate((hist_z_scores_x[0], hist_z_scores_y[0], hist_z_scores_z[0]), axis=None)
                
# Derivative vs differences (matlab code uses differences instead of derivative)
#                 differences = np.diff(rawdata,axis=0)
#                 # mean of differences
#                 mean_diff = np.asarray(np.mean(differences,axis=0))
#                 # std dev of differences
#                 std_diff = np.asarray(np.std(differences,axis=0))
#                 # skewness of differences
#                 skew_diff = skew(differences)
#                 # kurtosis of differences
#                 kurt_diff = kurtosis(differences)
                ##############################
                # derivative - for 3 axis
#                 derivative = difference/32
                derivativex = np.gradient(rawdata.iloc[:,0],32)
                derivativey = np.gradient(rawdata.iloc[:,1],32)
                derivativez = np.gradient(rawdata.iloc[:,2],32)
                derivative = np.array([derivativex,derivativey,derivativez])
                # derivative = np.gradient(rawdata, 32)
                # mean of derivative
                mean_derivative = np.mean(derivative,axis=1)
                # std dev of derivative
                std_derivative = np.std(derivative,axis=1)
                # skewness of derivative
                skew_derivative = skew(derivative,axis=1)
                # kurtosis of derivative
                kurt_derivative = kurtosis(derivative,axis=1)
#                 moments_of_derivative = np.array([mean_derivative,std_derivative,skew_derivative,kurt_derivative])
                moments_of_derivative = np.concatenate((mean_derivative,std_derivative,skew_derivative,kurt_derivative), axis=None)
                
                # sum of xyz std dev
#                 sum_stddev = np.std(rawdata.iloc[:,0] + np.std(rawdata.iloc[:,1]) + np.std(rawdata.iloc[:,2]))
                sum_stddev = np.array([np.std(rawdata.iloc[:,0] + np.std(rawdata.iloc[:,1]) + np.std(rawdata.iloc[:,2]))])
                
                # mean of the squared norm

# How should I get the Euclidiean norm?
#                 mean_of_squares = np.mean(np.mean(rawdata**2,axis=0)) # equation from matlab code - looks weird
# Is degree of freedom n (for mean) or (n-1)?
# Should be 1 feature
#                 norm = np.sqrt(np.square(rawdata).sum(axis=0))
#                 mean_squared_norm = np.mean(np.square(norm),axis=0)

                # Default axis=1, independently normalize each sample, otherwise (if 0) normalize each feature.
                norm = preprocessing.normalize(rawdata)
#                 mean_squared_norm = np.mean(np.square(norm))
                mean_squared_norm = np.array([np.mean(np.square(norm))])
    

#                 # normalize values (divided by acc norm) to get cross products
#                 norm = rawdata / np.linalg.norm(rawdata) # rawdata / np.linalg.norm(rawdata)
                # norm = rawdata/np.abs(np.sum(rawdata))
    

#                 # normalize values (divided by acc norm) to get cross products
#                 norm = rawdata / np.linalg.norm(rawdata) # rawdata / np.linalg.norm(rawdata)
                # norm = rawdata/np.abs(np.sum(rawdata))
#                 crossprod_norm_xy = np.nanmean(norm.iloc[:,0])*norm.iloc[:,1]
#                 crossprod_norm_xz = np.nanmean(norm.iloc[:,0])*norm.iloc[:,2]
#                 crossprod_norm_yz = np.nanmean(norm.iloc[:,1])*norm.iloc[:,2]
                crossprod_norm_xy = np.nanmean(norm[0]*norm[1])
                crossprod_norm_xz = np.nanmean(norm[0]*norm[2])
                crossprod_norm_yz = np.nanmean(norm[1]*norm[2])
                abs_crossprod_norm_xy = np.abs(crossprod_norm_xy)
                abs_crossprod_norm_xz = np.abs(crossprod_norm_xz)
                abs_crossprod_norm_yz = np.abs(crossprod_norm_yz)
                crossprod_raw_xy = np.nanmean(rawdata.iloc[:,0]*rawdata.iloc[:,1])
                crossprod_raw_xz = np.nanmean(rawdata.iloc[:,0]*rawdata.iloc[:,2])
                crossprod_raw_yz = np.nanmean(rawdata.iloc[:,1]*rawdata.iloc[:,2])
                abs_crossprod_raw_xy = np.abs(crossprod_raw_xy)
                abs_crossprod_raw_xz = np.abs(crossprod_raw_xz)
                abs_crossprod_raw_yz = np.abs(crossprod_raw_yz)
                crossprod = np.array([crossprod_raw_xy, crossprod_raw_xz, crossprod_raw_yz,
                                      crossprod_norm_xy, crossprod_norm_xz, crossprod_norm_yz,
                                      abs_crossprod_raw_xy, abs_crossprod_raw_xz, abs_crossprod_raw_yz,
                                      abs_crossprod_norm_xy, abs_crossprod_norm_xz, abs_crossprod_norm_yz])
                
                # sum of xyz std dev
#                 sum_stddev = np.std(rawdata.iloc[:,0] + np.std(rawdata.iloc[:,1]) + np.std(rawdata.iloc[:,2]))
                sum_stddev = np.array([np.std(rawdata.iloc[:,0] + np.std(rawdata.iloc[:,1]) + np.std(rawdata.iloc[:,2]))])
                
# What is this? Omit?               
#                 # Linear fit
#                 slope_xy = np.polyfit(rawdata.iloc[:,0],rawdata.iloc[:,1],1)[0]
#                 slope_xz = np.polyfit(rawdata.iloc[:,0],rawdata.iloc[:,2],1)[0]
#                 slope_yz = np.polyfit(rawdata.iloc[:,1],rawdata.iloc[:,2],1)[0]

                # power spectral density (PSD)
                Pxx = power_spectra_welch_axis(rawdata,fm=0,fM=10)
                #moments of PSD
#                 Pxx_moments = np.array([np.nanmean(Pxx.iloc[0].values),np.nanmean(Pxx.iloc[1].values),np.nanmean(Pxx.iloc[2].values),
#                                         np.nanstd(Pxx.iloc[0].values),np.nanstd(Pxx.iloc[1].values),np.nanstd(Pxx.iloc[2].values),
#                                         skew(Pxx.iloc[0].values),skew(Pxx.iloc[1].values),skew(Pxx.iloc[2].values),
#                                         kurtosis(Pxx.iloc[0].values),kurtosis(Pxx.iloc[1].values),kurtosis(Pxx.iloc[2].values)])
                Pxx_moments = np.array([np.nanmean(Pxx.iloc[:,0].values),np.nanmean(Pxx.iloc[:,1].values),np.nanmean(Pxx.iloc[:,2].values),
                        np.nanstd(Pxx.iloc[:,0].values),np.nanstd(Pxx.iloc[:,1].values),np.nanstd(Pxx.iloc[:,2].values),
                        skew(Pxx.iloc[:,0].values),skew(Pxx.iloc[:,1].values),skew(Pxx.iloc[:,2].values),
                        kurtosis(Pxx.iloc[:,0].values),kurtosis(Pxx.iloc[:,1].values),kurtosis(Pxx.iloc[:,2].values)])
                # Mean power in 0.5 Hz bins between 0 and 10 Hz (x, y, z)
                binedges = np.arange(0,10.5,0.5)
                powerbin_df = Pxx.groupby(pd.cut(Pxx.index, bins=binedges)).mean().fillna(0)
#                 powerbinarray = np.asarray((powerbin_df.iloc[:,0],powerbin_df.iloc[:,1],powerbin_df.iloc[:,2]))
                powerbinarray = np.concatenate((powerbin_df.iloc[:,0],powerbin_df.iloc[:,1],powerbin_df.iloc[:,2]), axis=None)
#####################################
############## End of added features
#####################################

                #Assemble features in array
# make sure all the columns add up and are labeled
#                 X = np.concatenate((mean,r,iqrange,std,sk,kurt,hist_z_scores,moments_of_derivative,mean_squared_norm,sum_stddev,
#                                     xcorr,crossprod,Pxx_moments,powerbinarray))
                X = np.concatenate((mean,r,iqrange,std,sk,kurt,hist_z_scores,moments_of_derivative,mean_squared_norm,sum_stddev,
                    xcorr,crossprod,Pxx_moments,powerbinarray))
                features.append(X)

            F = np.asarray(features) #feature matrix for all clips from current trial
        #     clip_data['features'] = pd.DataFrame(data=F,columns=features_list,dtype='float32')
            if sensor == 'accel': columns_list = acclist
            else: columns_list = gyrlist
            try:
                clip_data[trial][sensor]['features'] = pd.DataFrame(data=F,columns=columns_list,dtype='float32')
            except:
                print('Empty data in feature_extraction131')
                continue
#     return clip_data #not necessary